In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import IPython.display as display
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
import numpy as np
import pathlib
import pandas as pd
import pickle
from math import *
import os
from scipy import spatial
from scipy import linalg
from IPython.display import clear_output

In [2]:
predicateb = pd.read_csv('/home/gswaminathanpurdue/PredicateA.csv',header=None)
dum = predicateb.loc[:,1:]
dum.index = predicateb.loc[:,0]
predicateb = dum.iloc[1:,]

In [3]:
predicate = pd.read_csv('/home/gswaminathanpurdue/PredicateB.csv',header=None)
dum = predicate.loc[:,1:]
dum.index = predicate.loc[:,0]
predicate = dum.iloc[1:,]


In [4]:
# data_dir = tf.keras.utils.get_file(origin='https://cvml.ist.ac.at/AwA2/AwA2-data.zip',fname='Data', untar=True, extract = True, archive_format=True)

In [5]:
data_dir = '/home/gswaminathanpurdue/Data'
data_dir = pathlib.Path(data_dir)
data_dir

PosixPath('/home/gswaminathanpurdue/Data')

In [6]:
image_count = len(list(data_dir.glob('*/*.jpg')))

In [7]:
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"])
print(CLASS_NAMES)
class_names = list(CLASS_NAMES)

['skunk' 'weasel' 'humpback+whale' 'horse' 'sheep' 'bobcat' 'pig' 'deer'
 'german+shepherd' 'hamster' 'chihuahua' 'leopard' 'persian+cat' 'tiger'
 'grizzly+bear' 'hippopotamus' 'giant+panda' 'fox' 'squirrel' 'cow'
 'siamese+cat' 'dalmatian' 'blue+whale' 'ox' 'seal' 'moose' 'rat'
 'elephant' 'mole' 'gorilla' 'walrus' 'rhinoceros' 'wolf' 'otter' 'collie'
 'raccoon' 'bat' 'dolphin' 'mouse' 'zebra' 'spider+monkey' 'chimpanzee'
 'killer+whale' 'polar+bear' 'giraffe' 'buffalo' 'lion' 'beaver'
 'antelope' 'rabbit']


In [8]:
choose = np.random.choice(range(50),40,replace=False)
zslc = [x for x in range(0,50) if x not in choose]
cl1 = CLASS_NAMES[choose]
zslcl1 = CLASS_NAMES[zslc]

In [9]:
print('The labelled source data is:',cl1)
print('the unlabelled target data is:',zslcl1)

The labelled source data is: ['persian+cat' 'seal' 'polar+bear' 'rhinoceros' 'bat' 'blue+whale'
 'hamster' 'killer+whale' 'weasel' 'siamese+cat' 'beaver' 'hippopotamus'
 'leopard' 'wolf' 'zebra' 'chimpanzee' 'sheep' 'mouse' 'rabbit' 'cow'
 'skunk' 'spider+monkey' 'elephant' 'horse' 'ox' 'otter' 'raccoon'
 'giant+panda' 'dalmatian' 'buffalo' 'german+shepherd' 'tiger' 'rat'
 'deer' 'pig' 'walrus' 'moose' 'antelope' 'squirrel' 'collie']
the unlabelled target data is: ['humpback+whale' 'bobcat' 'chihuahua' 'grizzly+bear' 'fox' 'mole'
 'gorilla' 'dolphin' 'giraffe' 'lion']


In [10]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
BATCH_SIZE = 1000 # Change for different batch size
zslBATCH_SIZE = 500 # Change for different ZSL batch size
IMG_HEIGHT = 224
IMG_WIDTH = 224

In [11]:
train_data_gen = image_generator.flow_from_directory(directory=str(data_dir),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     classes = list(cl1))

Found 29761 images belonging to 40 classes.


In [12]:
zsl_data_gen = image_generator.flow_from_directory(directory=str(data_dir),
                                                     batch_size=zslBATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     classes = list(zslcl1))

Found 7561 images belonging to 10 classes.


ATTRIBUTE ACCURACY MEASUREMENT

In [13]:
def smatrix(Y,nam):
    sm =[]
    for n in range(len(Y)):
        a = nam[Y[n]==1][0]
        b = predicateb.loc[a,:]
        b = np.asarray(b,dtype=float).reshape(85,1)
        sm.append(b)
    sm = np.asarray(sm)
    return sm

In [14]:
print('Choose your model a) Inception b) Inception-Resnet c) Resnet')
a = str(input())

Choose your model a) Inception b) Inception-Resnet c) Resnet
Inception


In [15]:
input_tensor = Input(shape=(224,224,3))
if a=='Inception':
    model = InceptionV3(input_tensor=input_tensor,weights='imagenet',include_top=True)
elif a=='Resnet':
    model = ResNet50(input_tensor=input_tensor,weights='imagenet',include_top=True)
elif a=='Inception-Resnet':
    model = InceptionResNetV2(input_tensor=input_tensor,weights='imagenet',include_top=True)
else:
    print('Unspecified model, Please try again')

In [16]:
layer1 = Flatten()(model.layers[-3].output)
layer2 = Dense(85,activation=tf.keras.layers.ReLU(max_value=1))(layer1)
layer3 = Reshape((85,1))(layer2)
layer4 = Flatten()(layer3)
layer5 = Dense(40,activation='relu')(layer4)

In [17]:
new_model = Model([model.input],[layer2,layer5])

In [18]:
new_model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [19]:
def SAE(X,S,lamb):
    A=S.dot(S.T)
    B=lamb*(X.dot(X.T))
    C=(1+lamb)*(S.dot(X.T))
    D=linalg.solve_sylvester(A,B,C)
    return D

In [20]:
def NormalizeFea(Y):
    nS,mF = Y.shape
    feaN=np.sqrt(np.sum(Y*Y,1))
    dum=np.zeros((nS,mF))
    for ii in range(mF):
        dum[:,ii]=feaN
    res=Y/dum
    return res

In [21]:
def zmatrix(Y,nam):
    sm =[]
    for n in range(len(Y)):
        a = nam[Y[n]==1][0]
        b = predicate.loc[a,:]
        b = np.asarray(b,dtype=float).reshape(85,1)
        sm.append(b)
    sm = np.asarray(sm)
    return sm

In [22]:
def stegt(Y,one = True):
    ste = []
    for i in Y:
        if one:
            a = predicate.loc[i,:]
            a = np.asarray(a,dtype=float).reshape(85,1)
            ste.append(a)
        else:
            a = class_names.index(i)
            ste.append(a)
    ste = np.asarray(ste)
    return ste

In [23]:
def yte(Y,nam):
    yt = []
    for i in range(len(Y)):
        a = nam[Y[i]==1][0]
        a = class_names.index(a)
        yt.append(a)
    yt = np.asarray(yt)
    return yt  

In [24]:
getf = K.function([model.input],[model.layers[-2].output])
attribute_accuracy = []

In [25]:
batchX,batchY = next(train_data_gen)
Xtr = getf([batchX])[0]  
Xtr = NormalizeFea(Xtr.T).T
Xtr = np.nan_to_num(Xtr)  
Str = zmatrix(batchY,cl1)
Str = Str.reshape(Str.shape[0],Str.shape[1]) 

In [26]:
for itera in range(30):#change Iteration count to process more images
    clear_output(wait=True)  
    batchX,batchY = next(train_data_gen)
    batchS = smatrix(batchY,cl1)
    fitmodel = new_model.fit(batchX,[batchS,batchY],epochs=100, validation_split=0.2,shuffle=True,use_multiprocessing=True,verbose=0)
    gethl = K.function([new_model.input],[new_model.layers[-3].output])
    zslX,zslY = next(zsl_data_gen)
    shat = smatrix(zslY,zslcl1)  
    spred = gethl([zslX])[0]
    zsl_accuracy = []
    for j in range(len(spred)):
        count = 0
        for i in range(len(spred[j])):
            if (spred[j,i]==shat[j,i]):
                count+=1
            zsl_accuracy.append((count/85)*100)
    attribute_accuracy.append(np.mean(zsl_accuracy))
    batchX,batchY = next(train_data_gen)
    Xtr1 = getf([batchX])[0]  
    Xtr1 = NormalizeFea(Xtr1.T).T
    Xtr1 = np.nan_to_num(Xtr1)  
    Str1 = zmatrix(batchY,cl1)
    Str1 = Str1.reshape(Str1.shape[0],Str1.shape[1]) 
    Xtr = np.append(Xtr,Xtr1,axis=0)  
    Str = np.append(Str,Str1,axis=0)  
    print(itera)  

29


In [27]:
W = SAE(Xtr.T,Str.T,50000).T

ZSL Accuracy Measurement

In [28]:
zsl = []
for itera in range(11):
    zslX,zslY = next(zsl_data_gen)
    Xte = getf([zslX])[0]
    Xte = Xte.reshape(Xte.shape[0],Xte.shape[1])
    Stegt = stegt(zslcl1,True)
    Stegt = Stegt.reshape(Stegt.shape[0],Stegt.shape[1])
    Yte = yte(zslY,zslcl1)
    Yte = Yte.reshape(Yte.shape[0],1)
    teclid = stegt(zslcl1,False)
    teclid = teclid.reshape(teclid.shape[0],1)
    Steest = Xte .dot(NormalizeFea(W))
    dist = 1-spatial.distance.cdist(Steest,Stegt,'cosine')
    hits=1
    Yhit =np.zeros((dist.shape[0],hits))
    for i in range(dist.shape[0]):
        I=np.argsort(dist[i])[::-1]
        Yhit[i,:]=teclid[I[0:hits]]
    n=0
    for i in range(dist.shape[0]):
        if Yte[i] in Yhit[i,:]:
            n=n+1
    zsl_accuracy = n/dist.shape[0]
    print(zsl_accuracy)   
    zsl.append(zsl_accuracy)

0.718
0.704
0.704
0.71
0.684
0.724
0.716
0.762
0.724
0.686
0.72
